In [1]:
import gymnasium as gym
import numpy as np

env = gym.make("Taxi-v3")

# Recuperation du nombre d'etats et d'actions
state_size = env.observation_space.n
action_size = env.action_space.n

# Création de la table de politique initialisé de maniere aleatoire 
policy_table = np.random.rand(state_size, action_size)
# Normaliser pour que chaque ligne soit une distribution de probabilité
policy_table = policy_table / policy_table.sum(axis=1, keepdims=True)

# Création de la table de valeurs initialisee à zero
value_table = np.zeros(state_size)

# Affichage des premières lignes des tables
print("Policy Table Sample:")
print(policy_table[:5])
print("Value Table Sample:")
print(value_table[:5])

Policy Table Sample:
[[0.25195565 0.06782178 0.30795975 0.04566816 0.31407057 0.0125241 ]
 [0.21445019 0.15206801 0.08515017 0.17893825 0.24502397 0.12436941]
 [0.16688741 0.20249291 0.12133251 0.03363353 0.35364637 0.12200726]
 [0.16692999 0.21222052 0.00626109 0.20613276 0.1630647  0.24539094]
 [0.24654719 0.12391689 0.00187047 0.25848827 0.23918219 0.12999499]]
Value Table Sample:
[0. 0. 0. 0. 0.]


In [3]:
num_random_episodes = 20

print("\nExécution d'un agent aléatoire:")
for episode in range(num_random_episodes):
    state, _ = env.reset()
    done = False
    total_reward = 0
    actions_executed = []
    rewards_obtained = []
    
    while not done:
        action = env.action_space.sample()  # Action aléatoire
        next_state, reward, done, _, _ = env.step(action)
        
        actions_executed.append(action)
        rewards_obtained.append(reward)
        total_reward += reward
        state = next_state
    
    print(f"Épisode {episode + 1}:")
    print(f"Actions: {actions_executed}")
    print(f"Récompenses: {rewards_obtained}")
    print(f"Récompense totale: {total_reward}\n")


Exécution d'un agent aléatoire:
Épisode 1:
Actions: [np.int64(1), np.int64(5), np.int64(1), np.int64(5), np.int64(4), np.int64(3), np.int64(3), np.int64(4), np.int64(2), np.int64(2), np.int64(2), np.int64(3), np.int64(5), np.int64(3), np.int64(4), np.int64(2), np.int64(0), np.int64(4), np.int64(3), np.int64(1), np.int64(5), np.int64(1), np.int64(3), np.int64(3), np.int64(5), np.int64(3), np.int64(2), np.int64(4), np.int64(5), np.int64(3), np.int64(4), np.int64(0), np.int64(2), np.int64(1), np.int64(4), np.int64(0), np.int64(3), np.int64(4), np.int64(4), np.int64(4), np.int64(2), np.int64(5), np.int64(3), np.int64(1), np.int64(1), np.int64(0), np.int64(5), np.int64(4), np.int64(3), np.int64(4), np.int64(0), np.int64(2), np.int64(3), np.int64(5), np.int64(5), np.int64(3), np.int64(1), np.int64(1), np.int64(1), np.int64(5), np.int64(0), np.int64(2), np.int64(1), np.int64(3), np.int64(5), np.int64(3), np.int64(0), np.int64(1), np.int64(4), np.int64(1), np.int64(1), np.int64(3), np.int64(1

In [ ]:
def ppo_update(episodes, gamma=0.99, epsilon=0.2, alpha=0.01, lr=0.01):
    global policy_table, value_table
    
    for _ in range(episodes):
        # Collecter un épisode
        state, _ = env.reset()
        done = False
        episode_data = []
        
        while not done:
            action_probs = policy_table[state]
            action = np.random.choice(np.arange(action_size), p=action_probs)
            next_state, reward, done, _, _ = env.step(action)
            episode_data.append((state, action, reward))
            state = next_state
        
        # Calcul des récompenses cumulées (discounted rewards)
        states, actions, rewards = zip(*episode_data)
        discounted_rewards = []
        R = 0
        
        for r in reversed(rewards):
            R = r + gamma * R
            discounted_rewards.insert(0, R)
        
        # Normalisation des récompenses cumulées
        discounted_rewards = np.array(discounted_rewards)
        discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-9)
        
        # Mise à jour de la politique et des valeurs
        for i, (state, action, _) in enumerate(episode_data):
            # Calcul de l'avantage
            advantage = discounted_rewards[i] - value_table[state]
            
            # Ratio des politiques
            old_prob = policy_table[state, action]
            new_policy = policy_table.copy()
            new_policy[state] = np.clip(new_policy[state], 1e-10, 1.0 - 1e-10)  # Éviter les divisions par zéro
            new_policy[state] = new_policy[state] / new_policy[state].sum()  # Renormalisation
            new_prob = new_policy[state, action]
            ratio = new_prob / old_prob
            
            # Mise à jour avec clipping
            clipped_ratio = np.clip(ratio, 1 - epsilon, 1 + epsilon)
            policy_loss = -np.minimum(ratio * advantage, clipped_ratio * advantage)
            
            # Mise à jour de la politique
            policy_table[state, action] += lr * policy_loss
            
            # Renormalisation des probabilités
            policy_table[state] = np.clip(policy_table[state], 1e-10, 1.0 - 1e-10)
            policy_table[state] = policy_table[state] / policy_table[state].sum()
            
            # Mise à jour de la valeur
            value_table[state] += alpha * (discounted_rewards[i] - value_table[state])

# Entraînement avec PPO
print("\nDébut de l'entraînement avec PPO...")
ppo_update(episodes=100)  # 100 épisodes d'entraînement
print("Entraînement terminé!")


Début de l'entraînement avec PPO...


In [ ]:
num_eval_episodes = 20
total_rewards = []

print("\nÉvaluation de l'agent entraîné:")
for ep in range(num_eval_episodes):
    state, _ = env.reset()
    done = False
    total_reward = 0
    
    while not done:
        action_probs = policy_table[state]
        action = np.argmax(action_probs)  # Choix de l'action la plus probable
        next_state, reward, done, _, _ = env.step(action)
        total_reward += reward
        state = next_state
    
    total_rewards.append(total_reward)
    print(f"Épisode {ep + 1}: Récompense totale = {total_reward}")

# Comparaison des performances
print("\nRésumé des performances:")
print(f"Récompense moyenne avant entraînement (aléatoire): ~{env.reward_range[0]} à {env.reward_range[1]}")
print(f"Récompense moyenne après entraînement: {np.mean(total_rewards):.2f}")

env.close()